# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0b24619be0>
├── 'a' --> tensor([[-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457]])
└── 'x' --> <FastTreeValue 0x7f0b24619b20>
    └── 'c' --> tensor([[-0.1646,  1.9611,  0.0409, -1.3552],
                        [ 0.8999,  2.3325, -1.0056,  1.8081],
                        [ 0.2425,  1.0299, -0.9368,  0.8437]])

In [4]:
t.a

tensor([[-0.3126, -1.0108,  1.0250],
        [ 0.4771,  0.7780, -0.4457]])

In [5]:
%timeit t.a

63.1 ns ± 0.0352 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0b24619be0>
├── 'a' --> tensor([[-1.1890, -0.4388, -1.2816],
│                   [ 0.0095,  0.6457, -0.3663]])
└── 'x' --> <FastTreeValue 0x7f0b24619b20>
    └── 'c' --> tensor([[-0.1646,  1.9611,  0.0409, -1.3552],
                        [ 0.8999,  2.3325, -1.0056,  1.8081],
                        [ 0.2425,  1.0299, -0.9368,  0.8437]])

In [7]:
%timeit t.a = new_value

63.1 ns ± 0.0342 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3126, -1.0108,  1.0250],
               [ 0.4771,  0.7780, -0.4457]]),
    x: Batch(
           c: tensor([[-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437]]),
       ),
)

In [10]:
b.a

tensor([[-0.3126, -1.0108,  1.0250],
        [ 0.4771,  0.7780, -0.4457]])

In [11]:
%timeit b.a

59 ns ± 0.0159 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4172,  1.2879, -1.2413],
               [ 1.5867,  0.5134, -0.7597]]),
    x: Batch(
           c: tensor([[-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437]]),
       ),
)

In [13]:
%timeit b.a = new_value

483 ns ± 0.116 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

847 ns ± 0.261 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 34.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 658 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0b24619790>
├── 'a' --> tensor([[[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]],
│           
│                   [[-0.3126, -1.0108,  1.0250],
│                    [ 0.4771,  0.7780, -0.4457]]])
└── 'x' --> <FastTreeValue 0x7f0b24619490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 41 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0a79901940>
├── 'a' --> tensor([[-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457],
│                   [-0.3126, -1.0108,  1.0250],
│                   [ 0.4771,  0.7780, -0.4457]])
└── 'x' --> <FastTreeValue 0x7f0a7ff8c1c0>
    └── 'c' --> tensor([[-0.1646,  1.9611,  0.0409, -1.3552],
                        [ 0.8999,  2.3325, -1.0056,  1.8081],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 41.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 62.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1646,  1.9611,  0.0409, -1.3552],
                       [ 0.8999,  2.3325, -1.0056,  1.8081],
                       [ 0.2425,  1.0299, -0.9368,  0.8437]],
              
                      [[-0.1646,  1.9611,  0.0409, -1.3552],
                       [ 0.8999,  2.3325, -1.0056,  1.8081],
                       [ 0.2425,  1.0299, -0.9368,  0.8437]],
              
                      [[-0.1646,  1.9611,  0.0409, -1.3552],
                       [ 0.8999,  2.3325, -1.0056,  1.8081],
                       [ 0.2425,  1.0299, -0.9368,  0.8437]],
              
                      [[-0.1646,  1.9611,  0.0409, -1.3552],
                       [ 0.8999,  2.3325, -1.0056,  1.8081],
                       [ 0.2425,  1.0299, -0.9368,  0.8437]],
              
                      [[-0.1646,  1.9611,  0.0409, -1.3552],
                       [ 0.8999,  2.3325, -1.0056,  1.8081],
                       [ 0.2425,  1.0299, -0.9368,  0.8437]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437],
                      [-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437],
                      [-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437],
                      [-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437],
                      [-0.1646,  1.9611,  0.0409, -1.3552],
                      [ 0.8999,  2.3325, -1.0056,  1.8081],
                      [ 0.2425,  1.0299, -0.9368,  0.8437],
                      [-0.1646,  1.9611,  0.0409, -1.3552],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
